In [ ]:
import pythoncom
import pyfemap
import numpy as np
import pandas as pd
import plotly.graph_objs as go
import plotly.express as px

In [ ]:
# ensure model is open and active in Femap before running this cell
exist_obj = pythoncom.connect(pyfemap.model.CLSID)
app = pyfemap.model(exist_obj)

In [ ]:
def plot_mass_frac_from_femap(id_list, cumulative=False, min_frac=0.01):
    """Plots mode functions from Femap.

    Parameters
    ----------
    id_list : list of int
        list of Femap function IDs to plot
    cumulative : bool, optional
        Plots mass fractions by frequency if False. Plots cumulative sum of
        mass fractions, if True. Default is False
    min_frac : float, optional
        Minimum mass fraction to plot. All mass fractions lower than this
        minimum will be ignored and not plotted. Default is 0.01.

    Returns
    -------
    plotly.Figure
    """

    fn = app.feFunction
    df = pd.DataFrame()

    for id in id_list:
        fn.Get(id)
        data = np.array(fn.GetFunctionList()[2:])
        df[fn.title] = pd.Series(
            data=data[1],
            index=pd.Index(data[0], name='frequency')
        )

    fig = go.Figure(
        layout=go.Layout(
            title={
                'x': 0.5,
                'font': {'size': 20}
            },
            font={'family': 'Source Sans Pro',
                'size': 14},
            template='plotly_white',
            xaxis={'type': 'log',
                   'minor': {'dtick': 'D1'},
                   'showline': True,
                   'mirror': True,
                   'title': "Frequency (Hz)",
                   'range': [0, np.log10(2000)]},
            yaxis={'title': 'Mass Fraction',
                   'showline': True,
                   'mirror': True},
        )
    )

    if cumulative:
        for i in df.columns:
            dof = i[i.find('FRAC ') + 5:i.find('FRAC ') + 7]
            fig.add_scatter(
                x=df.index,
                y=df[i].cumsum(),
                name=dof,
                mode='lines',
                legendgroup='translation' if 'T' in dof else 'rotation'
            )
        fig.update_layout(
            title={'text': 'Cumulative Mass Fraction by Frequency'},
            hovermode='x'
        )
    else:
        for i in df.columns:
            dof = i[i.find('FRAC ') + 5:i.find('FRAC ') + 7]
            fig.add_scatter(
                x=df[df[i] > min_frac].index,
                y=df[df[i] > min_frac][i],
                name=dof,
                mode='markers',
                legendgroup='translation' if 'T' in dof else 'rotation',
                marker={'symbol': 'circle' if 'T' in dof else 'square',
                        'size': 9,
                        'line': {'width': 1.5}},
                hovertemplate="%{y:.2f}<br><b><i>%{x:.0f} Hz</i></b>"
            )
        fig.update_layout(
            title={
                'text': f"<b>Mass Fraction by Frequency</b><br>{min_frac} min"
            },
            xaxis={'showspikes': True,
                   'spikethickness': 1,
                   'spikemode': 'across'},
            hovermode='closest'
        )

    return fig

In [ ]:
# IDs of functions to plot
fn_list = list(range(1, 7))

In [ ]:
fig = plot_mass_frac_from_femap(fn_list)
fig.show()

In [ ]:
fig = plot_mass_frac_from_femap(fn_list, True)
fig.show()